**подключение библиотек**   
transforms - для быстрого умнлжения на матрицу Адамара

In [82]:
import numpy as np # type: ignore
import sys
import os

from scipy.linalg import qr # type: ignore
from scipy.linalg import svd # type: ignore
import numpy as np # type: ignore
import transforms # type: ignore

import time
import random

print(sys.executable)

/home/taisia/anaconda3/envs/rand_dec/bin/python


**my_matmul**
рукописное умножение для сравнений

In [83]:
def my_matmul(A, B):
  n, m = A.shape
  _, k = B.shape
  C = np.zeros((n, k))
  for i in range(0, n):
    for j in range(0, k):
      for l in range(0, m):
        C[i][j] += A[i][l] * B[l][j]
  return C

In [84]:
def matmul(A, B):
  return A @ B

**hadamard_projection**  
быстрое умножение матрицы на матрицу Адамара. возвращает случайные l столбцов

In [85]:
def hadamard_projection(A, k, p=5):
    m, n = A.shape
    l = k + p
    Y = A
    for i in range(0, m):
      Y[i, :] = transforms.fast_hadamard_transform(Y[i, :])
    random_cols = np.random.choice(n, l, replace = False)
    Y = Y[:, random_cols]
    return Y

### далее 3 функции описывают 3 разных подхода умножения матриц в алгоритме 4.4

- **randomized_subspace_iteration_with_hadamar**  
Алгоритм 4.4 из статьи.  
но для получения Y0 = A*Ω и самой случайной матрицы Ω используем hadamard_projection

In [86]:
def randomized_subspace_iteration_with_hadamar(A, k, p=3, q=3):
    Y0 = hadamard_projection(A, k, p)

    Q0, _ = np.linalg.qr(Y0)
    Qj = Q0
    for j in range(q):
        Y_tilde_j = A.conj().T @ Qj
        Q_tilde_j, _ = np.linalg.qr(Y_tilde_j)

        Yj = A @ Q_tilde_j
        Qj, _ = np.linalg.qr(Yj)

    return Qj

- **randomized_subspace_iteration**  
Честный алгоритм 4.4.  
Omega выбирается случайно, далее на нее умножается наша матрица. По умолчанию умножение **быстрое встроенное**.
 Если включен режим is_custom_mul, то происходит рукописное медленное умножение. (для сравнений результатов)

In [87]:
def randomized_subspace_iteration(A, k, p, q=3, is_custom_mul=False):
    _, n = A.shape
    l = k + p
    Omega = np.random.randn(n, l)
    if is_custom_mul:
        Y0 = my_matmul(A, Omega)
    else:
        Y0 = A @ Omega

    Q0, _ = np.linalg.qr(Y0)
    Qj = Q0
    for _ in range(q):
        Y_tilde_j = A.conj().T @ Qj
        Q_tilde_j, _ = np.linalg.qr(Y_tilde_j)

        Yj = A @ Q_tilde_j
        Qj, _ = np.linalg.qr(Yj)

    return Qj

### 2 функции подсчета loss  
1) использует встроенный mat_mul, честно считает ошибку
2) использует несколько случайных проекций. Возвращает максимальную ошибку.

- **get_loss**  
возвращает нормированную ошибку Q\*QT\*A - A.  
считает через встроенное умножение

In [88]:
def get_loss(A, Q):
  return np.linalg.norm(Q @ (Q.T @ A) - A) / np.linalg.norm(A)

In [89]:
accuracy = 1e-8

- **get_random_loss**  
    - get_array_projections.  
    первый шаг - формирование проекций
    - get_random_loss.  
    второй шаг - возврат максимальной ошибки

In [ ]:
def get_array_projections(A, seed=10):
    _, n = A.shape
    projections = list()
    for _ in range(seed):
        rand_vec = np.random.randn(n)
        projection = A @ rand_vec
        projections.append(projection / np.linalg.norm(projection))
    return projections

In [91]:
def get_random_loss(A, projections_arr, Q):
    max_loss = 0
    for projection in projections_arr:
        result = Q @ (Q.T @ projection)
        if np.linalg.norm(result - projection) > max_loss:
            max_loss = np.linalg.norm(result - projection)
    return max_loss
########## !!!!!!!!!!!!!!!!!!!!!!

In [92]:
def check_loss_rand(A, get_Q_func, projections_arr, k, p=3, q=3, max_loss = accuracy):
    Q = get_Q_func(A, k, p, q)
    loss = get_random_loss(A, projections_arr, Q)
    return loss < max_loss

In [93]:
def check_loss_k(A, get_Q_func, k, p=3, q=3, max_loss = accuracy):
    Q = get_Q_func(A, k, p, q)
    loss = get_loss(A, Q)
    return loss < max_loss

In [94]:
def get_rank_binary_search(A, left, right, get_Q_func, p=5, q=3, max_loss = accuracy):
  while right - left > 1: 
    k = (right + left) // 2
    if (check_loss_k(A, get_Q_func, k, p, q, max_loss)):
      right = k
    else:
      left = k
  return right

**get_rank**  
наш поиск ранга. 
1) ищет первую степень двойки, для которой ранг точно хорошо аппроксимирует 
2) бинарным поиском находим ранг  
проверка идет по check_loss_k.

In [95]:
def get_rank(A, get_Q_func, p=5, q=3, debug=False, max_loss=accuracy):
  k = 1
  while True:
    Q = get_Q_func(A, k, p, q)
    loss = get_loss(A, Q)
    if debug:
      print(f'now k = {k}, now loss = {loss}')
      
    if (check_loss_k(A, get_Q_func, k, p, q, max_loss)):
      return get_rank_binary_search(A, k // 2, k, get_Q_func, p, q, max_loss)
    k *= 2


In [96]:
def get_rand_rank(A, get_Q_func, p=5, q=3, seed=10, debug=False, max_loss=accuracy):
    projections_arr = get_array_projections(A, seed)
    k = 1
    while True:
        Q = get_Q_func(A, k, p, q)
        loss = get_loss(A, Q)
        if debug:
            print(f'now k = {k}, now loss = {loss}')
        if (check_loss_rand(A, get_Q_func, projections_arr, k, p, q, max_loss)):
            return get_rank_binary_search(A, k // 2, k, get_Q_func, p, q, max_loss)
        k *= 2

In [97]:
# def get_rank(A, get_Q, p=5, q=3, max_loss = 1e-14):
#   k = 1
#   while True:
#     Q = get_Q(A, k, p, q)
#     loss = get_loss(A, Q)
#     print(f'now k = {k}, now loss = {loss}')
#     if loss > max_loss:
#       k *= 2
#     else:
#       return k


In [98]:
def random_rank_k_matrix(m, n, k, random_state=None):
    B = np.random.randn(m, k)
    C = np.random.randn(k, n)
    A: np.matrix = B @ C
    return A

In [99]:
k = 45
p = 5
q = 3

In [100]:
m, n = 2**11, 2**11

In [101]:
A = random_rank_k_matrix(m, n, k)

In [102]:
def get_time(A, iteration):
    time_start = time.time()
    rank = get_rank(A, iteration)
    time_finish = time.time()
    return (time_finish - time_start, rank)

In [103]:
def get_rand_time(A, iteration, seed=10):
    time_start = time.time()
    rank = get_rand_rank(A, iteration, seed)
    time_finish = time.time()
    return (time_finish - time_start, rank)

In [104]:
def get_classic_time(A):
    time_start = time.time()
    rank = np.linalg.matrix_rank(A)
    time_finish = time.time()
    return (time_finish - time_start, rank)

In [105]:
time_start = time.time()
rank = get_rank(A, randomized_subspace_iteration_with_hadamar)
time_finish = time.time()
print(rank)
print(f'time = {time_finish - time_start}')
randomized_subspace_iteration_with_hadamar_time = time_finish - time_start

40
time = 4.348157644271851


In [106]:
# time_start = time.time()
# rank = get_rand_rank(A, randomized_subspace_iteration_with_hadamar)
# time_finish = time.time()
# print(rank)
# print(f'time = {time_finish - time_start}')
# randomized_subspace_iteration_with_hadamar_rand_time = time_finish - time_start

In [107]:
time_start = time.time()
rank = get_rank(A, randomized_subspace_iteration)
time_finish = time.time()
print(rank)
print(f'time = {time_finish - time_start}')
randomized_subspace_iteration_time = time_finish - time_start

40
time = 2.4892187118530273


In [108]:
time_start = time.time()
rank = get_rand_rank(A, randomized_subspace_iteration)
time_finish = time.time()
print(rank)
print(f'time = {time_finish - time_start}')
randomized_subspace_iteration_rand_time = time_finish - time_start

40
time = 1.9347078800201416


In [109]:
time_start = time.time()
np_rank = np.linalg.matrix_rank(A)
time_finish = time.time()
print(np_rank)
print(f'time = {time_finish - time_start}')
classic_rank_time = time_finish - time_start

45
time = 3.2907118797302246


In [110]:
print(randomized_subspace_iteration_with_hadamar_time)
# print(randomized_subspace_iteration_with_hadamar_rand_time)
print(randomized_subspace_iteration_time)
print(randomized_subspace_iteration_rand_time)
print(classic_rank_time)

4.348157644271851
2.4892187118530273
1.9347078800201416
3.2907118797302246


In [111]:
from tqdm import tqdm

all_time_subspace_rand = 0
all_time_classic = 0
for i in tqdm(range(5), desc="running"):
    
    all_time_classic += get_classic_time(A)[0]
    all_time_subspace_rand += get_rand_time(A, randomized_subspace_iteration)[0]
    
print(f"classic rank = {all_time_classic}")
print (f"our fast rank = {all_time_subspace_rand}")


running: 100%|██████████| 5/5 [00:29<00:00,  5.86s/it]

classic rank = 17.914172649383545
our fast rank = 11.383808851242065
